In [2]:
import pandas as pd

In [3]:
# Chargement du classement depuis le fichier CSV
data = pd.read_csv('data/france/ligue1-2024-2025.csv')

In [4]:
# Rename columns for easier handling
data.columns = ['POSITION', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS']
data.head()

,POSITION,Team,M,W,D,L,G,GA,PTS
0,1,Paris Saint Germain,28,23,5,0,80,26,74
1,2,Monaco,29,16,5,8,57,35,53
2,3,Marseille,29,16,4,9,57,41,52
3,4,Lyon,29,15,6,8,57,39,51
4,5,Lille,29,14,8,7,44,31,50


In [5]:
# League averages
league_avg_goals_for = data['G'].sum() / data['M'].sum()
league_avg_goals_against = data['GA'].sum() / data['M'].sum()

print(f"League Avg Goals Scored per Match: {league_avg_goals_for:.2f}")

League Avg Goals Scored per Match: 1.49


In [6]:
# Calculate team attack and defense strength
data['Attack_Strength'] = (data['G'] / data['M']) / league_avg_goals_for
data['Defense_Weakness'] = (data['GA'] / data['M']) / league_avg_goals_for

data[['Team', 'Attack_Strength', 'Defense_Weakness']].head()

,Team,Attack_Strength,Defense_Weakness
0,Paris Saint Germain,1.921944,0.624632
1,Monaco,1.322165,0.811856
2,Marseille,1.322165,0.951031
3,Lyon,1.322165,0.904639
4,Lille,1.020619,0.719072


In [22]:
def predict_match(team1, team2, data, max_goals=5, print_output=True):
    import numpy as np
    from scipy.stats import poisson

    # League average goals (you can also pass this in if you want to optimize)
    league_avg_goals = data['G'].sum() / data['M'].sum()

    # Get team stats
    t1 = data[data['Team'] == team1].iloc[0]
    t2 = data[data['Team'] == team2].iloc[0]

    # Expected goals using Poisson assumption
    exp_g1 = t1['Attack_Strength'] * t2['Defense_Weakness'] * league_avg_goals
    exp_g2 = t2['Attack_Strength'] * t1['Defense_Weakness'] * league_avg_goals

    # Poisson probability matrix
    prob_matrix = np.outer(
        [poisson.pmf(i, exp_g1) for i in range(max_goals + 1)],
        [poisson.pmf(j, exp_g2) for j in range(max_goals + 1)]
    )

    # Match outcome probabilities
    home_win_prob = np.tril(prob_matrix, -1).sum()
    draw_prob = np.trace(prob_matrix)
    away_win_prob = np.triu(prob_matrix, 1).sum()

    # Decimal odds (implied, without margin)
    home_odds = round(1 / home_win_prob, 2)
    draw_odds = round(1 / draw_prob, 2)
    away_odds = round(1 / away_win_prob, 2)

    if print_output:
        print(f"\n📊 Expected Goals:")
        print(f"{team1}: {exp_g1:.2f}, {team2}: {exp_g2:.2f}")

        print(f"\n📈 Match Outcome Probabilities:")
        print(f"{team1} Win: {home_win_prob:.2%}")
        print(f"Draw: {draw_prob:.2%}")
        print(f"{team2} Win: {away_win_prob:.2%}")

        print(f"\n🎯 Implied Decimal Odds:")
        print(f"{team1} Win: {home_odds}")
        print(f"Draw: {draw_odds}")
        print(f"{team2} Win: {away_odds}")

    return {
        'expected_goals': {team1: exp_g1, team2: exp_g2},
        'probabilities': {
            f'{team1}_win': home_win_prob,
            'draw': draw_prob,
            f'{team2}_win': away_win_prob,
        },
        'decimal_odds': {
            f'{team1}_win': home_odds,
            'draw': draw_odds,
            f'{team2}_win': away_odds,
        },
        'probability_matrix': prob_matrix
    }

In [23]:
predict_match('Paris Saint Germain', 'Le Havre', data)


📊 Expected Goals:
Paris Saint Germain: 4.11, Le Havre: 0.69

📈 Match Outcome Probabilities:
Paris Saint Germain Win: 68.93%
Draw: 5.44%
Le Havre Win: 2.41%

🎯 Implied Decimal Odds:
Paris Saint Germain Win: 1.45
Draw: 18.4
Le Havre Win: 41.45


{'expected_goals': {'Paris Saint Germain': np.float64(4.108983799705449),
  'Le Havre': np.float64(0.6892488954344624)},
 'probabilities': {'Paris Saint Germain_win': np.float64(0.6893315746601528),
  'draw': np.float64(0.05435199196796541),
  'Le Havre_win': np.float64(0.024126229023308556)},
 'decimal_odds': {'Paris Saint Germain_win': np.float64(1.45),
  'draw': np.float64(18.4),
  'Le Havre_win': np.float64(41.45)},
 'probability_matrix': array([[8.24430438e-03, 5.68237769e-03, 1.95828627e-03, 4.49915550e-04,
         7.75259490e-05, 1.06869349e-05],
        [3.38757131e-02, 2.33487979e-02, 8.04656657e-03, 1.84869571e-03,
         3.18552868e-04, 4.39124425e-05],
        [6.95973782e-02, 4.79699161e-02, 1.65316058e-02, 3.79813035e-03,
         6.54464288e-04, 9.02177575e-05],
        [9.53248332e-02, 6.57025360e-02, 2.26427002e-02, 5.20215203e-03,
         8.96394385e-04, 1.23567768e-04],
        [9.79220489e-02, 6.74926640e-02, 2.32596221e-02, 5.34388961e-03,
         9.20817502e-